# REGLAS DE ASOCIACIÓN

##### Autor: 
* Javier Tomás Fernández Martín

## 0. Preliminares

Antes de empezar con el código, importamos todas las librerias que vamos a necesitar.

In [187]:
import pandas as pd
import numpy as np
from apyori import apriori
from collections import Counter


Además de definir una semilla para asegurar que sea reproducible

In [15]:
random_state = 42


Y definimos un par de funciones que usaremos para evaluar módelos con distintos parámetros y compararlos entre sí

## 1.Carga de datos

In [16]:
df = pd.read_csv('bbdd.csv', index_col=0)


C:\Users\Javi\AppData\Local\Temp\ipykernel_7384\1186484719.py:1: DtypeWarning: Columns (3,4,39,40,42,43,44,92,112,150,151,152,242,243,257,258,259,266,267,269,270,271,390,391) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('bbdd.csv', index_col=0)


Mostramos una pequeña muestra para asegurar que se ha cargado bien y de paso echarle un vistazo a nuestra BBDD

In [17]:
df.sample(5, random_state=random_state)

,P1_TFT8_Ashe_Tier,P1_TFT8_Ashe_Obj1,P1_TFT8_Ashe_Obj2,P1_TFT8_Ashe_Obj3,P1_TFT8_Blitzcrank_Tier,P1_TFT8_Blitzcrank_Obj1,P1_TFT8_Blitzcrank_Obj2,P1_TFT8_Blitzcrank_Obj3,P1_TFT8_Galio_Tier,P1_TFT8_Galio_Obj1,...,P2_TFT8_Syndra_Obj2,P2_TFT8_Syndra_Obj3,P2_TFT8_Urgot_Tier,P2_TFT8_Urgot_Obj1,P2_TFT8_Urgot_Obj2,P2_TFT8_Urgot_Obj3,P2_Augment1,P2_Augment2,P2_Augment3,P1_Win
3883,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,TFT6_Augment_SecondWind1,TFT6_Augment_TradeSectorPlus,TFT8_Augment_KaisaCarry,1
2439,0,0,0,0,3,TFT_Item_BrambleVest,TFT_Item_IonicSpark,TFT8_Item_GenAEEmblemItem,0,0,...,TFT_Item_StatikkShiv,0,0,0,0,0,TFT8_Augment_HeartTrait,TFT8_Augment_AnnieSupport,TFT8_Augment_GenAEEmblem,0
1786,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,TFT6_Augment_SecondWind2,TFT6_Augment_Electrocharge2,TFT8_Augment_NunuSupport,1
2329,0,0,0,0,2,0,0,0,0,0,...,0,0,2,0,0,0,TFT6_Augment_ClearMind,TFT6_Augment_PortableForge,TFT8_Augment_ApheliosCarry,1
2567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,TFT6_Augment_GrandGambler,TFT7_Augment_Preparation2,TFT8_Augment_SonaExile,0


In [18]:

cat_cols = [col for col in df.columns if 'Obj' in col or 'Augment' in col]
df[cat_cols] = df[cat_cols].astype(str)

num_cols = [col for col in df.columns if col not in cat_cols + ['P1_Win']]
cat_cols_idx = [df.columns.get_loc(col) for col in cat_cols]


In [19]:
df_train,df_test = train_test_split(df, test_size=0.2, random_state=random_state)

## 2. Generación de modelos

In [20]:
# Transacciones
transactions = []
for i in range(len(df_train)):
    transaction = []
    for j in range(len(df_train.columns)):
        column_name = df_train.columns[j]
        if "_Tier" in column_name:
            champion_name = column_name.split("_Tier")[0]
            champion_tier = df_train.iloc[i][j]
            if champion_tier > 0:
                transaction.append(f"{champion_name}_Tier {champion_tier}")
        elif "_Obj" in column_name:
            item_name = column_name.split("_Obj")[0]
            item_value = df_train.iloc[i][j]
            if item_value != "0":
                transaction.append(f"{item_name} obj{column_name[-1]} {item_value}")
        elif "Augment" in column_name:
            augment_name = column_name.replace("_", "")
            augment_value = df_train.iloc[i][j]
            if augment_value != "0":
                transaction.append(f"{augment_name} {augment_value}")
    
    
    objective_value = str(df_train.iloc[i][-1])
    if objective_value == "0":
            transaction.append("P1_Lose")
    else:
            transaction.append("P1_Win")
            
    transactions.append(transaction)


In [184]:
train = transactions[0:(int(len(transactions)*0.7))]

test = transactions[int(len(transactions)*0.7):(len(transactions))]


In [186]:
# Generar las reglas de asociación
results = list(apriori(train, min_support=0.01, min_confidence=0.7))



In [188]:
# Imprimir las reglas de asociación
for r in results:
    print("{} -> {}: support={}, confidence={}, lift={}".format(
        list(r.ordered_statistics[0].items_base), list(r.ordered_statistics[0].items_add),
        r.support, r.ordered_statistics[0].confidence, r.ordered_statistics[0].lift))

['P1Augment1 TFT8_Augment_AnnieSupport'] -> ['P1_TFT8_Alistar_Tier 2']: support=0.012149141181399247, confidence=0.8787878787878788, lift=1.871245911388641
['P1Augment1 TFT8_Augment_AnnieSupport'] -> ['P1_TFT8_Annie_Tier 2']: support=0.012149141181399247, confidence=0.8787878787878788, lift=4.033974358974359
['P1Augment1 TFT8_Augment_EzrealSupport'] -> ['P1_TFT8_Ezreal_Tier 2']: support=0.011311269375785506, confidence=0.7941176470588236, lift=10.956987419245156
['P1Augment1 TFT8_Augment_EzrealSupport'] -> ['P1_TFT8_Kaisa_Tier 3']: support=0.010892333472978634, confidence=0.7647058823529411, lift=10.737370242214533
['P1Augment1 TFT8_Augment_GenAEEmblem'] -> ['P1_TFT8_Annie_Tier 2']: support=0.019271051529116047, confidence=0.8214285714285715, lift=3.7706730769230776
['P1Augment1 TFT8_Augment_GenAEEmblem'] -> ['P1_TFT8_Zoe_Tier 2']: support=0.018014243820695434, confidence=0.7678571428571429, lift=4.343305687203792
['P1Augment1 TFT8_Augment_LeeSinCarry'] -> ['P1_TFT8_Soraka_Tier 2']: su

In [189]:
final_rules=[]

for rule in results:
    if 'P1_Win' in rule.ordered_statistics[0].items_add:
        final_rules.append(rule)
    elif 'P1_Lose' in rule.ordered_statistics[0].items_add:
        final_rules.append(rule)

In [190]:
# Imprimir las reglas de asociación
for r in final_rules:
    print("{} -> {}: support={}, confidence={}, lift={}".format(
        list(r.ordered_statistics[0].items_base), list(r.ordered_statistics[0].items_add),
        r.support, r.ordered_statistics[0].confidence, r.ordered_statistics[0].lift))

['P1Augment1 TFT8_Augment_UndergroundTheTrait'] -> ['P1_Win']: support=0.012149141181399247, confidence=0.8529411764705883, lift=1.740145801910508
['P1_TFT8_Nilah obj1 TFT_Item_LocketOfTheIronSolari'] -> ['P1_Lose']: support=0.011730205278592375, confidence=0.7777777777777778, lift=1.5255181228887063
['P1_TFT8_Taliyah obj2 TFT_Item_JeweledGauntlet'] -> ['P1_Lose']: support=0.018014243820695434, confidence=0.7543859649122807, lift=1.4796378785913016
['P1_TFT8_Taliyah obj3 TFT_Item_SpearOfShojin'] -> ['P1_Lose']: support=0.01508169250104734, confidence=0.7826086956521738, lift=1.5349933907327353
['P2Augment2 TFT7_Augment_LastStand'] -> ['P1_Lose']: support=0.013824884792626729, confidence=0.75, lift=1.471035332785538
['P2Augment3 TFT8_Augment_ApheliosSupport'] -> ['P1_Lose']: support=0.010473397570171765, confidence=0.7575757575757576, lift=1.4858942755409477
['P2_TFT8_Aphelios obj3 TFT_Item_HextechGunblade'] -> ['P1_Lose']: support=0.01508169250104734, confidence=0.7058823529411764, lif

## 3. EVALUACIÓN DE MODELOS

Definimos una función que nos dice el número más común de una lista. Nos ayudará para el voto por la mayoría

In [191]:
def most_common_number(numbers):
    counter = Counter(numbers)
    most_common = counter.most_common(1)
    most_common_number = most_common[0][0]
    return most_common_number

Y se define una función para calcular el accuracy del sistema basado en reglas

In [192]:
def score_model(transactions, model_rules):
    correct_predictions = 0
    total_predictions = 0
    
    for transaction in transactions:
            
        rule_predictions = []
        
        #Se recorren todas las reglas y se comprueba cuales se disparan, anotando el resultado que predicen
        
        for rule in model_rules:
            triggered = True
            for item in rule[0]:
                if item == "P1_Lose" or item == "P1_Win":
                    pass
                elif item in transaction:
                    pass
                else:
                    triggered = False
                    
            if triggered:    
                if "P1_Lose" in rule[0]:
                    rule_predictions.append(0)
                if "P1_Win" in rule[0]:
                    rule_predictions.append(1)
            
            
        #Se comprueba que la lista no esté vacía, y si lo está se predice que pierde
        if rule_predictions != []:
            # Voto por mayoría
            prediction = most_common_number(rule_predictions)
        else:
            prediction = 0
            
        #Se hace la predicción teniendo en cuenta todas las reglas, y se comprueba si ha sido correcto o no.
        
        total_predictions += 1
        if (prediction == 1) and "P1_Win" in transaction :
            correct_predictions += 1
        elif (prediction == 0) and "P1_Lose" in transaction :
            correct_predictions += 1
        else:
            correct_predictions += 0
            
    
    if total_predictions > 0:
        
        accuracy = correct_predictions / total_predictions 
    else: 
        accuracy = 0
    

    return accuracy

In [196]:
score_model(train, final_rules)

0.6480938416422287

In [197]:
score_model(test, final_rules)

0.5865102639296188